In [55]:
from rapidfuzz import process, fuzz
import pandas as pd
import re
import numpy as np

In [56]:
#prepare loan dataset
PA_CD_raw = pd.read_excel("../../Data/Post1790/PA/PA_post1790_CD.xlsx",
                      header = 11, usecols = 'G, H, I, J, K, L, M, U, V, W, X, Y, Z, AA, AJ, AK, AL, AM, AN, AO, AP')
PA_CD_raw.columns = ['First Name', 'Last Name', 'town1', 'state1', 'occupation1', '6p_Dollar', '6p_Cents',
                 'First Name.1', 'Last Name.1', 'town2', 'state2', 'occupation2', '6p_def_Dollar', '6p_def_Cents',
                 'First Name.2', 'Last Name.2', 'town3', 'state3', 'occupation3', '3p_Dollar', '3p_Cents', ]
PA_CD_raw['Name'] = PA_CD_raw['First Name'].apply(lambda x: "" if pd.isnull(x) else x) + " " + PA_CD_raw['Last Name'].apply(lambda x: "" if pd.isnull(x) else x)
PA_CD_raw['Name'] = PA_CD_raw['Name'].apply(lambda x: re.sub(' +', ' ', x).strip())
PA_CD_raw['occupation1'] = PA_CD_raw['occupation1'].apply(lambda x: "" if pd.isnull(x) else x)

In [57]:
grouped_df = PA_CD_raw.groupby(['Name', 'town1', 'occupation1']).agg({'6p_Dollar': ['sum', 'count']}).reset_index()
grouped_df.columns = ['Name', 'Town', 'occupation', '6p_total', '6p_count']
grouped_df = grouped_df.sort_values('6p_total', ascending = False).reset_index(drop = True)
grouped_df['pct_cum'] = grouped_df['6p_total'].cumsum()/grouped_df['6p_total'].sum()
grouped_df['pct'] = grouped_df['6p_total']/grouped_df['6p_total'].sum()

In [58]:
grouped_df

,Name,Town,occupation,6p_total,6p_count,pct_cum,pct
0,Mordecai Lewis,Philadelphia,Merchant,88152.0,12,0.057169,0.057169
1,Blair McClenahan,Philadelphia,,62341.0,3,0.097598,0.040430
2,William Morris and Swanwick,Philadelphia,Merchant,54009.0,6,0.132624,0.035026
3,George Eddy,Philadelphia,Merchant,29630.0,12,0.151840,0.019216
4,Robert Buchanan,Philadelphia,Merchant,25633.0,8,0.168463,0.016624
...,...,...,...,...,...,...,...
694,David Hahn,York Town,,3.0,1,0.999995,0.000002
695,Mark Wilcocks,Delaware County,,3.0,1,0.999997,0.000002
696,Silas Engles,Southwark,,3.0,1,0.999999,0.000002
697,James Bell,Allegany County,,2.0,1,1.000000,0.000001


In [59]:
loan = pd.read_csv('../../Data/Pre1790/cleaned/loan_office_certificates_9_states_cleaned.csv', index_col = 0)
marine = pd.read_csv('../../Data/Pre1790/cleaned/Marine_Liquidated_Debt_Certificates_cleaned.csv', index_col = 0)
pierce = pd.read_csv('../../Data/Pre1790/cleaned/Pierce_Certs_cleaned_2021.csv', index_col = 0)
liquidCT = pd.read_csv('../../Data/Pre1790/cleaned/liquidated_debt_certificates_CT_cleaned.csv', index_col = 0)
liquidDE = pd.read_csv('../../Data/Pre1790/cleaned/liquidated_debt_certificates_DE_cleaned.csv', index_col = 0)
liquidMA = pd.read_csv('../../Data/Pre1790/cleaned/liquidated_debt_certificates_MA_cleaned.csv', index_col = 0)
liquidNJ = pd.read_csv('../../Data/Pre1790/cleaned/liquidated_debt_certificates_NJ_cleaned.csv', index_col = 0)
liquidNH = pd.read_csv('../../Data/Pre1790/cleaned/liquidated_debt_certificates_NH_cleaned.csv', index_col = 0)
liquidNY = pd.read_csv('../../Data/Pre1790/cleaned/liquidated_debt_certificates_NY_cleaned.csv', index_col = 0)
liquidPA1 = pd.read_csv('../../Data/Pre1790/cleaned/liquidated_debt_certificates_PA_story_cleaned.csv', index_col = 0)
liquidRI = pd.read_csv('../../Data/Pre1790/cleaned/liquidated_debt_certificates_RI_cleaned.csv', index_col = 0)
liquidPA2 = pd.read_csv('../../Data/Pre1790/cleaned/liquidated_debt_certificates_PA_stelle_cleaned.csv', index_col = 0)

C:\Users\david\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [60]:
def nameLiquidated(fname, lname):
    ctsum = liquidCT[[ln in lname and fn in fname for ln, fn in zip(liquidCT['Last name'],
                                                                    liquidCT['First name'])]]['Dollars'].sum()
    desum = liquidDE[[ln in lname and fn in fname for ln, fn in zip(liquidDE['Last name'],
                                                                    liquidDE['First name'])]]['Dollars'].sum()
    masum = liquidMA[[ln in lname and fn in fname for ln, fn in zip(liquidMA['Last name'],
                                                                    liquidMA['First name'])]]['Dollars'].sum()
    njsum = liquidNJ[[ln in lname and fn in fname for ln, fn in zip(liquidNJ['Last name'],
                                                                    liquidNJ['First name'])]]['Dollars'].sum()
    nhsum = liquidNH[[ln in lname and fn in fname for ln, fn in zip(liquidNH['Last name'],
                                                                    liquidNH['First name'])]]['Dollars'].sum()
    pasum1 = liquidPA1[[ln in lname and fn in fname for ln, fn in zip(liquidPA1['Last name'],
                                                                      liquidPA1['First name'])]]['Dollars'].sum()
    risum = liquidRI[[ln in lname and fn in fname for ln, fn in zip(liquidRI['Last name'],
                                                                    liquidRI['First name'])]]['Dollars'].sum()
    pasum2 = liquidPA2[[ln in lname and fn in fname for ln, fn in zip(liquidPA2['Last name'],
                                                                      liquidPA2['First name'])]]['Dollars'].sum()
    return ctsum + desum + masum + njsum + nhsum + pasum1 + risum + pasum2

## David Grier

In [61]:
pierce[[ln == 'Grier' and fn in ['David'] for ln, fn in zip(pierce['Last'],
                                                                           pierce['First'])]]['Value'].sum()

0.0

In [62]:
marine[[ln == 'Grier' and fn in ['David'] for ln, fn in zip(marine['Last name'],
                                                                           marine['First name'])]]['Dollars'].sum()

0.0

In [63]:
loan[[ln == 'Grier' and fn in ['David'] for ln, fn in zip(loan['Last Name 1 '],
                                                                        loan['First Name 1 '])]]['Face Value'].sum()

0

In [64]:
nameLiquidated(['David'], ['Grier'])

196.0

## John Schweighauser

In [65]:
pierce[[ln == 'Schweighauser' and fn in ['John'] for ln, fn in zip(pierce['Last'],
                                                                pierce['First'])]]['Value'].sum()

0.0

In [66]:
marine[[ln == 'Schweighauser' and fn in ['John'] for ln, fn in zip(marine['Last name'],
                                                                marine['First name'])]]['Dollars'].sum()

0.0

In [67]:
loan[[ln == 'Schweighauser' and fn in ['John'] for ln, fn in zip(loan['Last Name 1 '],
                                                              loan['First Name 1 '])]]['Face Value'].sum()

8000

In [68]:
nameLiquidated(['John'], ['Schweighauser'])

0.0

## Maria Benezet (Guardian: Daniel Benezett)

In [69]:
marine[[ln == 'Benezett' and fn in ['Daniel'] for ln, fn in zip(marine['Last name'],
                                                                                        marine['First name'])]]['Dollars'].sum()

0.0

In [70]:
pierce[[ln == 'Benezett' and fn in ['Daniel'] for ln, fn in zip(pierce['Last'],
                                                                                        pierce['First'])]]['Value'].sum()

0.0

In [71]:
loan[[ln == 'Benezett' and fn in ['Daniel'] for ln, fn in zip(loan['Last Name 1 '],
                                                                                      loan['First Name 1 '])]]['Face Value'].sum()

6000

In [72]:
nameLiquidated(['Maria'], ['Benezet'])

0.0

## Job Vernon

In [73]:
marine[[ln == 'Vernon' and fn in ['Job'] for ln, fn in zip(marine['Last name'],
                                                                                        marine['First name'])]]['Dollars'].sum()

0.0

In [74]:
pierce[[ln == 'Vernon' and fn in ['Job'] for ln, fn in zip(pierce['Last'],
                                                                                        pierce['First'])]]['Value'].sum()

2994.34

In [75]:
loan[[ln == 'Vernon' and fn in ['Job'] for ln, fn in zip(loan['Last Name 1 '],
                                                                                      loan['First Name 1 '])]]['Face Value'].sum()

0

In [76]:
nameLiquidated(['John'], ['Vernon'])

0.0

## Adam Guier

In [77]:
marine[[ln == 'Guier' and fn in ['Adam'] for ln, fn in zip(marine['Last name'],
                                                                                        marine['First name'])]]['Dollars'].sum()

0.0

In [78]:
pierce[[ln == 'Guier' and fn in ['Adam'] for ln, fn in zip(pierce['Last'],
                                                                                        pierce['First'])]]['Value'].sum()

0.0

In [79]:
loan[[ln == 'Guier' and fn in ['Adam'] for ln, fn in zip(loan['Last Name 1 '],
                                                             loan['First Name 1 '])]]['Face Value'].sum()

0

In [80]:
nameLiquidated(['Adam'], ['Guier'])

0.0

## William Honeyman

In [81]:
pierce[[ln == 'Honeyman' and fn in ['William'] for ln, fn in zip(pierce['Last'],
                                                                                        pierce['First'])]]['Value'].sum()

2699.2499999999995

In [82]:
marine[[ln == 'Honeyman' and fn in ['William'] for ln, fn in zip(marine['Last name'],
                                                           marine['First name'])]]['Dollars'].sum()

0.0

In [83]:
loan[[ln == 'Honeyman' and fn in ['William'] for ln, fn in zip(loan['Last Name 1 '],
                                                         loan['First Name 1 '])]]['Face Value'].sum()

0

In [84]:
nameLiquidated(['William'], ['Honeyman'])

0.0

## William Chevalier (Guardian: Mary Chevalier)

Looked into his supposed father, John Chevalier.

In [85]:
pierce[[ln == 'Chevalier' and fn in ['John'] and 'Pennsylvania' in issue for ln, fn, issue in zip(pierce['Last'],
                                                                  pierce['First'],
                                                                  pierce['To Whom Issued'])]]['Value'].sum()

0.0

In [86]:
marine[[ln == 'Chevalier' and fn in ['John'] for ln, fn in zip(marine['Last name'],
                                                           marine['First name'])]]['Dollars'].sum()

0.0

In [87]:
loan[[ln == 'Chevalier' and fn in ['John'] for ln, fn in zip(loan['Last Name 1 '],
                                                         loan['First Name 1 '])]]['Face Value'].sum()

38500

In [88]:
nameLiquidated(['John'], ['Chevalier'])

0.0